In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/L03_keyframes.zip /content

In [3]:
!unzip -q 'L03_keyframes.zip'

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import svd
from PIL import Image
import os
import glob
import time
from tqdm import tqdm

In [5]:
%cd  '/content/L03_keyframes'

/content/L03_keyframes


In [7]:
# Image compression using SVD

def compress_image(img_name, k):
    #print("processing...")
    global compressed_image
    img = np.asarray(Image.open(img_name))

    r = img[:,:,0]
    g = img[:,:,1]
    b = img[:,:,2]

    #print("compressing...")
    ur,sr,vr = svd(r, full_matrices=False)
    ug,sg,vg = svd(g, full_matrices=False)
    ub,sb,vb = svd(b, full_matrices=False)
    rr = np.dot(ur[:,:k],np.dot(np.diag(sr[:k]), vr[:k,:]))
    rg = np.dot(ug[:,:k],np.dot(np.diag(sg[:k]), vg[:k,:]))
    rb = np.dot(ub[:,:k],np.dot(np.diag(sb[:k]), vb[:k,:]))

    #print("arranging...")
    rimg = np.zeros(img.shape)
    rimg[:,:,0] = rr
    rimg[:,:,1] = rg
    rimg[:,:,2] = rb

    for ind1, row in enumerate(rimg):
        for ind2, col in enumerate(row):
            for ind3, value in enumerate(col):
                if value < 0:
                    rimg[ind1,ind2,ind3] = abs(value)
                if value > 255:
                    rimg[ind1,ind2,ind3] = 255

    compressed_image = rimg.astype(np.uint8)
    return compressed_image

# Resize img

def resize_img(img, size=(255,144)):
      img = Image.fromarray(img)
      img = img.resize(size)
      return img

In [10]:
import torch
import torch.nn.functional as F
from PIL import Image
import numpy as np

def compress_image(img_name, k):
    global compressed_image

    # Load the image and move it to GPU
    img = np.asarray(Image.open(img_name))
    img_tensor = torch.tensor(img, dtype=torch.float32).cuda()

    # Split the image into RGB channels and move to GPU
    r = img_tensor[:, :, 0]
    g = img_tensor[:, :, 1]
    b = img_tensor[:, :, 2]

    # Perform SVD on each channel using GPU-accelerated torch.svd
    ur, sr, vr = torch.svd(r, some=True)
    ug, sg, vg = torch.svd(g, some=True)
    ub, sb, vb = torch.svd(b, some=True)

    # Compress the channels on GPU
    rr = torch.matmul(ur[:, :k], torch.matmul(torch.diag(sr[:k]), vr[:, :k].t()))
    rg = torch.matmul(ug[:, :k], torch.matmul(torch.diag(sg[:k]), vg[:, :k].t()))
    rb = torch.matmul(ub[:, :k], torch.matmul(torch.diag(sb[:k]), vb[:, :k].t()))

    # Create the compressed image tensor
    rimg = torch.zeros_like(img_tensor)
    rimg[:, :, 0] = rr
    rimg[:, :, 1] = rg
    rimg[:, :, 2] = rb

    # Ensure pixel values are within the valid range
    rimg = torch.clamp(rimg, 0, 255)

    # Convert the compressed image back to a NumPy array
    compressed_image = rimg.cpu().numpy().astype(np.uint8)
    return compressed_image

def resize_img(img, size=(255,144)):
    img = Image.fromarray(img)
    img = img.resize(size)
    return img


In [11]:
# Main
os.makedirs('compressed_L03_V001', exist_ok=True)  # Need to change
folder_name = 'L03_V001' # Need to change
for i in tqdm(sorted(os.listdir(folder_name))):
      compressed_img = compress_image(folder_name + '/' + i, 100)
      img = resize_img(compressed_img)
      img.save('compressed_L03_V001/' + i[:-3] + 'webp', 'webp')

100%|██████████| 1173/1173 [05:17<00:00,  3.69it/s]


In [ ]:
# Compare folder size after compression
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / 1000000

In [ ]:
folder_path1 = '/content/drive/MyDrive/preprocess_img/L03_keyframes/L03_V001'
folder_path2 = '/content/drive/MyDrive/preprocess_img/L03_keyframes/compressed_L03_V001'
print(f"Dung lượng của thư mục ban đầu là {get_folder_size(folder_path1): .2f} MB\nDung lượng của thư mục sau khi nén là {get_folder_size(folder_path2): .2f} MB")

Dung lượng của thư mục ban đầu là  225.46 MB
Dung lượng của thư mục sau khi nén là  7.51 MB
